In [2]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [ ]:
cat > test_vroom.R << 'EOF'
### set environment
cat("\n++++++++++ Set environment  ++++++++++\n")
#source("/home/mount/project/config_sing.R")
source("config_sing.R")

cat("\n++++++++++ Set global variables ++++++++++\n")
### Get argument: Chromomsome
ARGS        = commandArgs(trailingOnly=TRUE)
CHROM       = ARGS[1]
IS_INPUT20X = as.logical(ARGS[2])
FDIRY       = ARGS[3]
THRESHOLD   = 10

### set global variables
SAMPLES_TOT = c(
    paste0("Input", 1:5),
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

SAMPLES_INP = c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

SAMPLES_INP20X = c(
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

if (IS_INPUT20X) {
    SAMPLES = SAMPLES_INP20X
    FDIRY   = paste0(FDIRY, "_", "input20x")
} else {
    SAMPLES = SAMPLES_INP
}
FD_OUT = file.path(FD_RES, "model_linear", FDIRY, CHROM)

### print start message
cat("Chromosome:      ", CHROM,       "\n")
cat("Is Input20x used?", IS_INPUT20X, "\n")
cat("Output Directory:", FD_OUT,      "\n")

###################################################
### Import library size
###################################################
cat("\n++++++++++ Import library size ++++++++++\n")

### Helper function to get
get_group = function(idn_sample){
    idn = idn_sample
    
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    
    idn = str_remove(
        string = idn, 
        pattern = "_20x")
    
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "FPath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)

### remove the total size
dat_lib = dat_lib %>% dplyr::filter(FPath != "total")

### summarize info from the file path
### stackoverflow: Extract only folder name right before filename from full path
dat_lib = dat_lib %>% 
    #mutate(Sample = tools::file_path_sans_ext(basename(FPath))) %>%
    mutate(Sample = basename(dirname(FPath))) %>%
    mutate(Group = get_group(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)
print(dat_lib)

###################################################
### Import annotated fragments
###################################################
cat("\n++++++++++ Import annotated fragments ++++++++++\n")

### set column names and types
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

### import bed files for each sample 
fdiry = file.path(FD_RES, "annotation_fragment", "filter_motif_score095")
fname = paste0(CHROM, ".bed.gz")

lst_dat = lapply(SAMPLES_TOT, function(sam){
    ### set path
    fpath = file.path(fdiry, sam, fname)
    print(fpath); flush.console()
    
    ### import data
    dat = vroom(fpath, col_types=ctypes, col_names=cnames) %>% mutate(Sample = sam)
    return(dat)
})

### arrange data
dat_ann_frag = bind_rows(lst_dat)
lst_dat      = NULL

### check environment
cat("Current memory used after import data:\n")
mem_used()

EOF

In [ ]:
cat > test_readtsv.R << 'EOF'
### set environment
cat("\n++++++++++ Set environment  ++++++++++\n")
#source("/home/mount/project/config_sing.R")
source("config_sing.R")

cat("\n++++++++++ Set global variables ++++++++++\n")
### Get argument: Chromomsome
ARGS        = commandArgs(trailingOnly=TRUE)
CHROM       = ARGS[1]
IS_INPUT20X = as.logical(ARGS[2])
FDIRY       = ARGS[3]
THRESHOLD   = 10

### set global variables
SAMPLES_TOT = c(
    paste0("Input", 1:5),
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

SAMPLES_INP = c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

SAMPLES_INP20X = c(
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

if (IS_INPUT20X) {
    SAMPLES = SAMPLES_INP20X
    FDIRY   = paste0(FDIRY, "_", "input20x")
} else {
    SAMPLES = SAMPLES_INP
}
FD_OUT = file.path(FD_RES, "model_linear", FDIRY, CHROM)

### print start message
cat("Chromosome:      ", CHROM,       "\n")
cat("Is Input20x used?", IS_INPUT20X, "\n")
cat("Output Directory:", FD_OUT,      "\n")

###################################################
### Import library size
###################################################
cat("\n++++++++++ Import library size ++++++++++\n")

### Helper function to get
get_group = function(idn_sample){
    idn = idn_sample
    
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    
    idn = str_remove(
        string = idn, 
        pattern = "_20x")
    
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "FPath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)

### remove the total size
dat_lib = dat_lib %>% dplyr::filter(FPath != "total")

### summarize info from the file path
### stackoverflow: Extract only folder name right before filename from full path
dat_lib = dat_lib %>% 
    #mutate(Sample = tools::file_path_sans_ext(basename(FPath))) %>%
    mutate(Sample = basename(dirname(FPath))) %>%
    mutate(Group = get_group(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)
print(dat_lib)

###################################################
### Import annotated fragments
###################################################
cat("\n++++++++++ Import annotated fragments ++++++++++\n")

### set column names and types
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

### import bed files for each sample 
fdiry = file.path(FD_RES, "annotation_fragment", "filter_motif_score095")
fname = paste0(CHROM, ".bed.gz")

lst_dat = lapply(SAMPLES_TOT, function(sam){
    ### set path
    fpath = file.path(fdiry, sam, fname)
    print(fpath); flush.console()
    
    ### import data
    dat = read_tsv(fpath, col_types=ctypes, col_names=cnames) %>% mutate(Sample = sam)
    return(dat)
})

### arrange data
dat_ann_frag = bind_rows(lst_dat)
lst_dat      = NULL

### check environment
cat("Current memory used after import data:\n")
mem_used()

EOF

In [8]:
ls -lh ${FD_RES}/annotation_fragment/filter_motif_score095/Input1_20x

total 5.9G
-rw-rw-r-- 1 kk319 reddylab 303M Oct  2 15:27 chr10.bed.gz
-rw-rw-r-- 1 kk319 reddylab 315M Oct  2 15:26 chr11.bed.gz
-rw-rw-r-- 1 kk319 reddylab 270M Oct  2 15:22 chr12.bed.gz
-rw-rw-r-- 1 kk319 reddylab 130M Oct  2 15:14 chr13.bed.gz
-rw-rw-r-- 1 kk319 reddylab 179M Oct  2 15:16 chr14.bed.gz
-rw-rw-r-- 1 kk319 reddylab 180M Oct  2 15:16 chr15.bed.gz
-rw-rw-r-- 1 kk319 reddylab 294M Oct  2 15:19 chr16.bed.gz
-rw-rw-r-- 1 kk319 reddylab 301M Oct  2 15:22 chr17.bed.gz
-rw-rw-r-- 1 kk319 reddylab 120M Oct  2 15:11 chr18.bed.gz
-rw-rw-r-- 1 kk319 reddylab 317M Oct  2 15:14 chr19.bed.gz
-rw-rw-r-- 1 kk319 reddylab 552M Oct  2 16:05 chr1.bed.gz
-rw-rw-r-- 1 kk319 reddylab 200M Oct  2 15:16 chr20.bed.gz
-rw-rw-r-- 1 kk319 reddylab  73M Oct  2 15:05 chr21.bed.gz
-rw-rw-r-- 1 kk319 reddylab 190M Oct  2 15:09 chr22.bed.gz
-rw-rw-r-- 1 kk319 reddylab 438M Oct  2 15:42 chr2.bed.gz
-rw-rw-r-- 1 kk319 reddylab 335M Oct  2 15:35 chr3.bed.gz
-rw-rw-r-- 1 kk319 reddylab 248M Oct  2 15:27 ch

In [6]:
tail -n 5 ${FD_LOG}/test_memory_chrom_30G.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_30G.19.txt <==
srun: error: dl-01: task 0: Out Of Memory

Done!
Run Time: 6 minutes and 50 seconds
slurmstepd: error: Detected 1 oom-kill event(s) in StepId=26542096.batch cgroup. Some of your processes may have been killed by the cgroup out-of-memory handler.

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_30G.20.txt <==
Current memory used after import data:
10.3 GB

Done!
Run Time: 3 minutes and 0 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_30G.21.txt <==
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_DMSO/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX2_Dex/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX3_Dex/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_mot

In [7]:
tail -n 5 ${FD_LOG}/test_memory_chrom_60G.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_60G.19.txt <==
Current memory used after import data:
28.7 GB

Done!
Run Time: 7 minutes and 53 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_60G.20.txt <==
Current memory used after import data:
10.3 GB

Done!
Run Time: 2 minutes and 3 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_60G.21.txt <==
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_DMSO/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX2_Dex/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX3_Dex/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX4_Dex/chr22.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_Dex/chr22.bed.gz"

==> /gpfs/fs1/

In [3]:
tail ${FD_LOG}/test_memory_chrom_70G.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_70G.19.txt <==
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_DMSO/chr20.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX2_Dex/chr20.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX3_Dex/chr20.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX4_Dex/chr20.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_Dex/chr20.bed.gz"
Current memory used after import data:
28.7 GB

Done!
Run Time: 5 minutes and 37 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/test_memory_chrom_70G.20.txt <==
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_DMSO/chr21.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TF